In [1]:
import pandas as pd
from collections import Counter

# 1. Load data
df_players = pd.read_csv("sampled_segmentation.csv")
df_topics = pd.read_csv("game_top_topics_labeled.csv")

# 2. Normalisasi nama game untuk pencocokan yang lebih baik
df_players['Game Name'] = df_players['Game Name'].str.lower().str.strip()
df_topics['game'] = df_topics['game'].str.lower().str.strip()

# 3. Gabungkan topik ke data pemain
df_merged = df_players.merge(df_topics[['game', 'Top_Topic_Labels']],
                              left_on='Game Name', right_on='game', how='left')

# 4. Konversi string list menjadi list Python asli (jika masih berbentuk string)
df_merged['Top_Topic_Labels'] = df_merged['Top_Topic_Labels'].apply(
    lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [])

# 5. Hitung topik dominan untuk tiap Steam ID
def get_dominant_topic(topics_list):
    flat_list = [topic for topics in topics_list for topic in topics]
    if flat_list:
        return Counter(flat_list).most_common(1)[0][0]
    return None

dominant_topics = (
    df_merged.groupby('Steam ID')['Top_Topic_Labels']
    .agg(get_dominant_topic)
    .reset_index()
    .rename(columns={'Top_Topic_Labels': 'Dominant Topic'})
)

# 6. Gabungkan kembali ke dataframe awal
df_final = df_players.merge(dominant_topics, on='Steam ID', how='left')

# 7. Simpan hasil
df_final.to_csv("segmentation_with_dominant_topic.csv", index=False)
